In [2]:
!pip install scikit-learn
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 50.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.5 MB/s eta 0:00:00eta 0:00:01


In [3]:
import os
import rasterio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [4]:
train_files = ['tile00.csv', 'tile01.csv', 'tile02.csv' ,'tile03.csv']
train = pd.concat([pd.read_csv(f) for f in train_files], ignore_index=True)
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)



/tmp/ipykernel_3431/3763426186.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

In [5]:
# Train the Random Forest classifier
clf = RandomForestClassifier(n_estimators=30, random_state=42, verbose=2)
clf.fit(X_train, y_train)


building tree 1 of 30
building tree 2 of 30
building tree 3 of 30
building tree 4 of 30
building tree 5 of 30
building tree 6 of 30
building tree 7 of 30
building tree 8 of 30
building tree 9 of 30
building tree 10 of 30
building tree 11 of 30
building tree 12 of 30
building tree 13 of 30
building tree 14 of 30
building tree 15 of 30
building tree 16 of 30
building tree 17 of 30
building tree 18 of 30
building tree 19 of 30
building tree 20 of 30
building tree 21 of 30
building tree 22 of 30
building tree 23 of 30
building tree 24 of 30
building tree 25 of 30
building tree 26 of 30
building tree 27 of 30
building tree 28 of 30
building tree 29 of 30
building tree 30 of 30


RandomForestClassifier(n_estimators=30, random_state=42, verbose=2)

In [6]:
predictions = clf.predict(X_test)


In [7]:

print(classification_report(y_test, predictions))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   3138409
         1.0       0.86      0.20      0.32       487
         2.0       0.90      0.22      0.35       279
         3.0       1.00      1.00      1.00         1
         4.0       0.76      0.20      0.32       185
         5.0       0.75      0.14      0.23        65
         6.0       0.88      0.15      0.26        46
         7.0       0.00      0.00      0.00        10

    accuracy                           1.00   3139482
   macro avg       0.77      0.36      0.43   3139482
weighted avg       1.00      1.00      1.00   3139482



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
print(np.unique(predictions), np.unique(y_test), np.unique(y_train))

[0. 1. 2. 3. 4. 5. 6.] [0. 1. 2. 3. 4. 5. 6. 7.] [0. 1. 2. 3. 4. 5. 6. 7.]


## Hyperparamter Tuning with GridSearch

In [6]:
from sklearn.model_selection import RandomizedSearchCV
rs_space={'max_depth':[None, 3],
              'n_estimators':[10,20],
              'max_features':[1, 5, 10],
              'criterion':['gini','entropy'],
              'min_samples_leaf':[1, 2, 3, 4],
              'min_samples_split':[2, 5]
          }

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(rf, rs_space, n_iter=50, n_jobs=-1, cv=3, verbose=1)


In [7]:

model_random = rf_random.fit(X_train, y_train)

# random random search results
print('Best random search hyperparameters are: '+str(model_random.best_params_))
print('Best random search score is: '+str(model_random.best_score_))

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best random search hyperparameters are: {'n_estimators': 10, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 10, 'max_depth': None, 'criterion': 'entropy'}
Best random search score is: 0.9996869819862434


In [8]:
predictions = rf_random.predict(X_test)
print(classification_report(y_test, predictions))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   3138409
         1.0       0.87      0.16      0.26       487
         2.0       0.94      0.21      0.35       279
         3.0       1.00      1.00      1.00         1
         4.0       0.80      0.15      0.25       185
         5.0       0.75      0.18      0.30        65
         6.0       0.90      0.20      0.32        46
         7.0       0.00      0.00      0.00        10

    accuracy                           1.00   3139482
   macro avg       0.78      0.36      0.44   3139482
weighted avg       1.00      1.00      1.00   3139482



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
# predictions = clf.predict(X_test)
# print(classification_report(y_test, predictions))